In [1]:
import time
import pandas as pd
import numpy as np
import warnings
from collections import Counter
warnings.filterwarnings('ignore')
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
pd.set_option('display.max_columns', None)


There are many insignificant features to our work, like codes, PK, FK and out of context. 
Based on prior works and experience of especialists, will be taken off nonsense features. 
A study was carried out on the evolution and correspondence of these resources in the ENEM and Census
data for the period 2009-2018. See (featuresOrganisaton.xls)

In [2]:
classmate = ['ANO_CENSO','PK_COD_ENTIDADE','PK_COD_TURMA', 'FK_COD_MOD_ENSINO', 'FK_COD_ETAPA_ENSINO', 'NUM_MATRICULAS']

teacher = ['ANO_CENSO','FK_COD_DOCENTE', 'PK_COD_ENTIDADE','PK_COD_TURMA','ID_ESPECIALIZACAO','ID_MESTRADO','ID_DOUTORADO',
          'ID_QUIMICA', 'ID_FISICA', 'ID_MATEMATICA', 'ID_BIOLOGIA', 'ID_SOCIOLOGIA', 'ID_LINGUA_LITERAT_PORTUGUESA',
           'ID_LINGUA_LITERAT_INGLES', 'ID_LINGUA_LITERAT_ESPANHOL', 'ID_ARTES', 'ID_EDUCACAO_FISICA', 'ID_HISTORIA',
          'ID_GEOGRAFIA','ID_FILOSOFIA','FK_COD_AREA_OCDE_1', 'ID_LICENCIATURA_1', 'FK_COD_AREA_OCDE_2',
           'ID_LICENCIATURA_2','FK_COD_AREA_OCDE_3', 'ID_LICENCIATURA_3']
school = ['ANO_CENSO','PK_COD_ENTIDADE', 'FK_COD_MUNICIPIO','FK_COD_ESTADO','ID_DEPENDENCIA_ADM','ID_AGUA_INEXISTENTE',
                  'ID_ESGOTO_INEXISTENTE','ID_ENERGIA_INEXISTENTE',
                   'ID_LABORATORIO_INFORMATICA','ID_LABORATORIO_CIENCIAS','ID_SALA_ATENDIMENTO_ESPECIAL',
                   'ID_BIBLIOTECA','ID_SALA_LEITURA','ID_SANITARIO_DENTRO_PREDIO',
                   'ID_SANITARIO_PNE','NUM_SALAS_UTILIZADAS','NUM_EQUIP_TV',
                   'NUM_EQUIP_DVD','NUM_EQUIP_COPIADORA','NUM_EQUIP_IMPRESSORA',
                   'NUM_COMP_ALUNOS','ID_INTERNET','ID_BANDA_LARGA','NUM_FUNCIONARIOS','ID_ALIMENTACAO', 
                   'ID_REG_MEDIO_MEDIO','ID_REG_MEDIO_INTEGRADO','ID_REG_MEDIO_NORMAL', 'ID_SALA_PROFESSOR', 
                   'ID_QUADRA_ESPORTES_COBERTA','ID_QUADRA_ESPORTES_DESCOBERTA','ID_COZINHA','NUM_EQUIP_PARABOLICA', 
          'ID_MOD_ATIV_COMPLEMENTAR'] 

enem = ['NU_ANO','COD_ESCOLA','IN_TP_ENSINO','IDADE','TP_SEXO','TP_COR_RACA','ST_CONCLUSAO','NOTA_CN',
        'NOTA_CH','NOTA_LC','NOTA_MT','NU_NOTA_REDACAO','Q001','Q002','Q004','Q003','ID_DEPENDENCIA_ADM_ESC'
]


In [3]:
#Students of ENEM
E = pd.read_csv('~/data/enem/2014/DADOS/MICRODADOS_ENEM_2014.csv', sep=',', encoding="iso-8859-2", usecols = enem)
print(E.shape)
E.drop_duplicates('COD_ESCOLA').shape

(8722248, 17)


(32306, 17)

In [4]:
#How many public schools (not included federal) are in the databases?
mun_est = (E.ID_DEPENDENCIA_ADM_ESC==2) |(E.ID_DEPENDENCIA_ADM_ESC==3)
print('students at public schools:',(mun_est).sum())
print('public schools:',(E.drop_duplicates('COD_ESCOLA')[mun_est]['COD_ESCOLA'].count()))
E.drop('ID_DEPENDENCIA_ADM_ESC', axis=1,inplace=True)
enem.remove('ID_DEPENDENCIA_ADM_ESC')

students at public schools: 1344736
public schools: 22846


In [5]:
#School census data
CE =    pd.read_csv("~/data/censo/2014/DADOS/ESCOLAS.CSV", sep='|', encoding="iso-8859-2", usecols = school)

In [6]:
#Teacher census data by Braazilian mesoregion
CD_CO = pd.read_csv("~/data/censo/2014/DADOS/DOCENTES_CO.CSV", sep='|', encoding="iso-8859-2", usecols = teacher)
CD_NE = pd.read_csv("~/data/censo/2014/DADOS/DOCENTES_NORDESTE.CSV", sep='|', encoding="iso-8859-2", usecols = teacher)
CD_N =  pd.read_csv("~/data/censo/2014/DADOS/DOCENTES_NORTE.CSV", sep='|', encoding="iso-8859-2", usecols = teacher)
CD_SE = pd.read_csv("~/data/censo/2014/DADOS/DOCENTES_SUDESTE.CSV", sep='|', encoding="iso-8859-2", usecols = teacher)
CD_S =  pd.read_csv("~/data/censo/2014/DADOS/DOCENTES_SUL.CSV", sep='|', encoding="iso-8859-2", usecols = teacher)

#All teacher census data
CT = pd.concat([CD_CO, CD_NE, CD_N, CD_SE, CD_S])

In [7]:
#ClassMate census data
CM =  pd.read_csv("~/data/censo/2014/DADOS/TURMAS.CSV", sep='|', encoding="iso-8859-2", usecols = classmate)

In [8]:
ENEM = E.copy()
C_SCHOOL = CE.copy()
C_TEACHER = CT.copy()
C_CLASS = CM.copy()

In [9]:
#ordering
ENEM = ENEM[enem]

newNames = [
'CO_ANO',
'CO_ESCOLA',
'TP_ENSINO',
'NU_IDADE',
'TP_SEXO',
'TP_COR_RACA',
'TP_ST_CONCLUSAO',
'NU_NOTA_CN',
'NU_NOTA_CH',
'NU_NOTA_LC',
'NU_NOTA_MT',
'NU_NOTA_REDACAO',
'EDU_PAI',
'EDU_MAE',
'QT_PESSOAS_CASA',
'RENDA_MENSAL'
]

#Rename
ENEM.columns = newNames

print('checking year',ENEM.CO_ANO.unique())
print('checking key',ENEM.CO_ESCOLA.dtypes)

checking year [2014]
checking key float64


In [10]:
C_SCHOOL = C_SCHOOL[school]

newNames = ['ANO_CENSO',
'CO_ESCOLA',
 'CO_MUNICIPIO',
 'CO_UF',
 'CO_DEPENDENCIA_ADM',
 'IN_AGUA_INEXISTENTE',
 'IN_ESGOTO_INEXISTENTE',
 'IN_ENERGIA_INEXISTENTE',
 'IN_LABORATORIO_INFORMATICA',
 'IN_LABORATORIO_CIENCIAS',
 'IN_SALA_ATENDIMENTO_ESPECIAL',
 'IN_BIBLIOTECA',
 'IN_SALA_LEITURA',
 'IN_BANHEIRO',
 'IN_BANHEIRO_PNE',
 'QT_SALAS_UTILIZADAS',
 'QT_EQUIP_TV',
 'QT_EQUIP_DVD',
 'QT_EQUIP_COPIADORA',
 'QT_EQUIP_IMPRESSORA',
 'QT_COMP_ALUNO',
 'IN_INTERNET',
 'IN_BANDA_LARGA',
 'QT_FUNCIONARIOS',
 'IN_ALIMENTACAO',
 'IN_COMUM_MEDIO_MEDIO',
 'IN_COMUM_MEDIO_INTEGRADO',
 'IN_COMUM_MEDIO_NORMAL', 
 'IN_SALA_PROFESSOR', 
 'IN_QUADRA_ESPORTES_COBERTA',
 'IN_QUADRA_ESPORTES_DESCOBERTA',           
 'IN_COZINHA',
 'IN_EQUIP_PARABOLICA',
 'IN_ATIV_COMPLEMENTAR',
]

#Rename 
C_SCHOOL.columns =  newNames

#Adapting for all years
C_SCHOOL.loc[((C_SCHOOL.IN_QUADRA_ESPORTES_COBERTA ==1) | (C_SCHOOL.IN_QUADRA_ESPORTES_DESCOBERTA ==1)), 'IN_QUADRA_ESPORTES'] = 1 
C_SCHOOL.drop(['IN_QUADRA_ESPORTES_COBERTA', 'IN_QUADRA_ESPORTES_DESCOBERTA'], inplace= True, axis=1)

print('Checking year of censo',C_SCHOOL.ANO_CENSO.unique())
C_SCHOOL.drop('ANO_CENSO', axis=1, inplace=True)
print('checking key',C_SCHOOL.CO_ESCOLA.dtypes)

Checking year of censo [2014]
checking key int64


In [11]:
C_TEACHER = C_TEACHER[teacher]

# New default column names for all Datasets
newNames = [
'ANO_CENSO',
'CO_PROFESSOR',
'CO_ESCOLA',  
'CO_TURMA',
'IN_ESPECIALIZACAO',
'IN_MESTRADO',
'IN_DOUTORADO',
'ID_QUIMICA',
'ID_FISICA',
'ID_MATEMATICA',
'ID_BIOLOGIA',
'ID_SOCIOLOGIA',
'ID_LINGUA_LITERAT_PORTUGUESA',
'ID_LINGUA_LITERAT_INGLES',
'ID_LINGUA_LITERAT_ESPANHOL',
'ID_ARTES',
'ID_EDUCACAO_FISICA',
'ID_HISTORIA',
'ID_GEOGRAFIA',
'ID_FILOSOFIA',
'CO_AREA_OCDE_1', 
'ID_LICENCIATURA_1', 
'CO_AREA_OCDE_2',
'ID_LICENCIATURA_2',
'CO_AREA_OCDE_3',
'ID_LICENCIATURA_3'
]

C_TEACHER.columns = newNames
print('Checking year of censo',C_TEACHER.ANO_CENSO.unique())
C_TEACHER.drop('ANO_CENSO', axis=1, inplace=True)
print('checking key',C_TEACHER.CO_ESCOLA.dtypes)

Checking year of censo [2014]
checking key int64


In [12]:
C_CLASS = C_CLASS[classmate]

newNames = [
'ANO_CENSO',    
'CO_ESCOLA',
'CO_TURMA',  
'CO_MOD_ENSINO',
'IN_ETAPA_ENSINO',
'QT_MATRICULAS'
]

#Rename 
C_CLASS.columns =  newNames

print('Checking year of censo',C_CLASS.ANO_CENSO.unique())
C_CLASS.drop('ANO_CENSO', axis=1, inplace=True)
print('checking key',C_CLASS.CO_ESCOLA.dtypes)

Checking year of censo [2014]
checking key int64


### Minimum Scope Definition

ENEM  - Taking off:
* Without School Code
* Is not attended in regular schools, 
*  With scores zero
*  between 17 and 19 years old (in order to take only last secodary year students)
* Attended in schools with at least 10 students

In [13]:
ENEM.dropna(inplace = True, axis=0, subset = ['CO_ESCOLA'])


In [14]:
#should care about the nulls into scope filter features?
filters = (['TP_ST_CONCLUSAO', 'TP_ENSINO'])
ENEM[filters].isnull().sum()
#NO, IT'S LOWER

TP_ST_CONCLUSAO       0
TP_ENSINO          7942
dtype: int64

In [15]:
print('drop out',(ENEM.TP_ENSINO!= 1).sum(), 'who not attend regular and prof approachs schools')
ENEM = ENEM.loc[(ENEM.TP_ENSINO== 1)]
ENEM.drop('TP_ENSINO', inplace = True, axis =1)


drop out 202231 who not attend regular and prof approachs schools


In [16]:
fields = (['NU_NOTA_CN', 'NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT', 'NU_NOTA_REDACAO'])
for i in fields:
    print ('null:', i, ENEM[i].isnull().sum())
    print ('zero:', i, (ENEM[i]== 0).sum())
    #, "and", ENEM.I==0)

null: NU_NOTA_CN 200506
zero: NU_NOTA_CN 376
null: NU_NOTA_CH 200506
zero: NU_NOTA_CH 137
null: NU_NOTA_LC 220600
zero: NU_NOTA_LC 761
null: NU_NOTA_MT 220600
zero: NU_NOTA_MT 1209
null: NU_NOTA_REDACAO 0
zero: NU_NOTA_REDACAO 279478


In [17]:
ENEM.dropna(inplace = True, axis=0, subset=fields)
ENEM = ENEM.loc[~(ENEM[fields] == 0).any(axis=1)]

In [18]:
#ONLY STUDENTS AT  17 -19 YEAR OLD. WE WANT TO GET ONLY REGULAR STUDENTS THAT ARE IN LAST YEAR OF SECONDARY SCHOOL. 
#THE MICRODADOS HAVE A FIELD TO CONTROL IT. BUT IT IS NOT CONFIDENCE,DUE A TURN OUT TEAACHERS THAT EXTEND ACADEMIC YEAR
print('null', ENEM.NU_IDADE.isnull().sum())
#firts, fill NA with the mean of student school
ENEM.NU_IDADE = ENEM.groupby(ENEM['CO_ESCOLA'])['NU_IDADE'].apply(lambda x: x.fillna(round(x.mean())))
ENEM.NU_IDADE = ENEM.NU_IDADE.astype('int32')
print('taking off',((ENEM.NU_IDADE <17)|(ENEM.NU_IDADE>19)).sum())
ENEM = ENEM.loc[(ENEM.NU_IDADE >16) &  (ENEM.NU_IDADE<20)]
ENEM.NU_IDADE.value_counts()

null 64
taking off 133566


17    668457
18    365770
19    102991
Name: NU_IDADE, dtype: int64

In [19]:
ENEM[fields].describe()

,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO
count,1.137218e+06,1.137218e+06,1.137218e+06,1.137218e+06,1.137218e+06
mean,4.924933e+02,5.528157e+02,5.179764e+02,4.878535e+02,5.199815e+02
std,7.522545e+01,7.166336e+01,6.689689e+01,1.108958e+02,1.531279e+02
min,3.420000e+02,3.302000e+02,3.062000e+02,3.185000e+02,4.000000e+01
25%,4.358000e+02,5.071000e+02,4.772000e+02,4.037000e+02,4.000000e+02
50%,4.849000e+02,5.582000e+02,5.222000e+02,4.652000e+02,5.200000e+02
75%,5.419000e+02,6.008000e+02,5.633000e+02,5.527000e+02,6.200000e+02
max,8.764000e+02,8.621000e+02,8.090000e+02,9.736000e+02,1.000000e+03


In [20]:
#Only schools with at least 10 students 
filter = ENEM.loc[ENEM.groupby('CO_ESCOLA')['CO_ESCOLA'].transform('count')<=9]
print('taking off',filter.shape[0], 'students of', filter.drop_duplicates('CO_ESCOLA').shape[0], 'schools')
ENEM = ENEM.loc[~ENEM.CO_ESCOLA.isin(filter.CO_ESCOLA)]

taking off 27067 students of 6328 schools


In [21]:
#Classes on Census 
#Only regular classes
C_CLASS = C_CLASS.loc[C_CLASS['CO_MOD_ENSINO']==1]
#Only third year of secondary school
step_class = [27,28,32,33,37,38] # 
C_CLASS = C_CLASS.loc[C_CLASS['IN_ETAPA_ENSINO'].isin(step_class)]
C_CLASS.shape[0]

77801

### Treatment

ENEM

In [22]:
fields = ['TP_SEXO', 'TP_COR_RACA', 'EDU_PAI', 'EDU_MAE','QT_PESSOAS_CASA', 'RENDA_MENSAL']
ENEM[fields].isnull().sum()

TP_SEXO            0
TP_COR_RACA        0
EDU_PAI            0
EDU_MAE            0
QT_PESSOAS_CASA    0
RENDA_MENSAL       0
dtype: int64

In [23]:
for i in fields:
    print(i,ENEM[i].value_counts().index)

TP_SEXO Index(['F', 'M'], dtype='object')
TP_COR_RACA Int64Index([1, 3, 2, 4, 0, 5], dtype='int64')
EDU_PAI Index(['E', 'B', 'C', 'G', 'I', 'D', 'F', 'H', 'A'], dtype='object')
EDU_MAE Index(['E', 'C', 'B', 'G', 'D', 'H', 'F', 'I', 'A'], dtype='object')
QT_PESSOAS_CASA Int64Index([4, 3, 5, 6, 2, 7, 8, 1, 9, 10, 11, 12, 13, 15, 14, 20, 16, 17, 18,
            19],
           dtype='int64')
RENDA_MENSAL Index(['B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'Q', 'A', 'K', 'O', 'L',
       'N', 'M', 'P'],
      dtype='object')


In [24]:
#Order relation, is better for numeric categories.
categorical = ['TP_SEXO','EDU_PAI', 'EDU_MAE', 'RENDA_MENSAL']
for i in categorical:
    codes, uniques = pd.factorize(ENEM[i], sort=True)
    ENEM[i] = codes

In [25]:
# Paring with default categories

ENEM.loc[ENEM['EDU_PAI']==0, 'EDU_PAI_TEMP']=0
ENEM.loc[ENEM['EDU_PAI']==1, 'EDU_PAI_TEMP']=1
ENEM.loc[ENEM['EDU_PAI']==2, 'EDU_PAI_TEMP']=2
ENEM.loc[ENEM['EDU_PAI']==3, 'EDU_PAI_TEMP']=2
ENEM.loc[ENEM['EDU_PAI']==4, 'EDU_PAI_TEMP']=3
ENEM.loc[ENEM['EDU_PAI']==5, 'EDU_PAI_TEMP']=3
ENEM.loc[ENEM['EDU_PAI']==6, 'EDU_PAI_TEMP']=4
ENEM.loc[ENEM['EDU_PAI']==7, 'EDU_PAI_TEMP']=5
ENEM.loc[ENEM['EDU_PAI']==8, 'EDU_PAI_TEMP']=0


ENEM.loc[ENEM['EDU_MAE']==0, 'EDU_MAE_TEMP']=0
ENEM.loc[ENEM['EDU_MAE']==1, 'EDU_MAE_TEMP']=1
ENEM.loc[ENEM['EDU_MAE']==2, 'EDU_MAE_TEMP']=2
ENEM.loc[ENEM['EDU_MAE']==3, 'EDU_MAE_TEMP']=2
ENEM.loc[ENEM['EDU_MAE']==4, 'EDU_MAE_TEMP']=3
ENEM.loc[ENEM['EDU_MAE']==5, 'EDU_MAE_TEMP']=3
ENEM.loc[ENEM['EDU_MAE']==6, 'EDU_MAE_TEMP']=4
ENEM.loc[ENEM['EDU_MAE']==7, 'EDU_MAE_TEMP']=5
ENEM.loc[ENEM['EDU_MAE']==8, 'EDU_MAE_TEMP']=0


ENEM.loc[ENEM['QT_PESSOAS_CASA']==1, 'QT_PESSOAS_CASA_TEMP']=0
ENEM.loc[ENEM['QT_PESSOAS_CASA']==2, 'QT_PESSOAS_CASA_TEMP']=1
ENEM.loc[ENEM['QT_PESSOAS_CASA']==3, 'QT_PESSOAS_CASA_TEMP']=1
ENEM.loc[ENEM['QT_PESSOAS_CASA']==4, 'QT_PESSOAS_CASA_TEMP']=2
ENEM.loc[ENEM['QT_PESSOAS_CASA']==5, 'QT_PESSOAS_CASA_TEMP']=2
ENEM.loc[ENEM['QT_PESSOAS_CASA']==6, 'QT_PESSOAS_CASA_TEMP']=2
ENEM.loc[ENEM['QT_PESSOAS_CASA']==7, 'QT_PESSOAS_CASA_TEMP']=2
ENEM.loc[ENEM['QT_PESSOAS_CASA']>7, 'QT_PESSOAS_CASA_TEMP']=3

ENEM.loc[ENEM['RENDA_MENSAL']==0, 'RENDA_MENSAL_TEMP']=0
ENEM.loc[ENEM['RENDA_MENSAL']==1, 'RENDA_MENSAL_TEMP']=1
ENEM.loc[ENEM['RENDA_MENSAL']==2, 'RENDA_MENSAL_TEMP']=2
ENEM.loc[ENEM['RENDA_MENSAL']==3, 'RENDA_MENSAL_TEMP']=2
ENEM.loc[(ENEM['RENDA_MENSAL']>3) & (ENEM['RENDA_MENSAL']<8), 'RENDA_MENSAL_TEMP']=3
ENEM.loc[(ENEM['RENDA_MENSAL']>7) & (ENEM['RENDA_MENSAL']<13), 'RENDA_MENSAL_TEMP']=4
ENEM.loc[ENEM['RENDA_MENSAL']>12, 'RENDA_MENSAL_TEMP']=5

In [26]:
#back to original columns
ENEM['EDU_PAI'] = ENEM['EDU_PAI_TEMP']
ENEM['EDU_MAE'] = ENEM['EDU_MAE_TEMP']
ENEM['QT_PESSOAS_CASA'] = ENEM['QT_PESSOAS_CASA_TEMP']
ENEM['RENDA_MENSAL'] = ENEM['RENDA_MENSAL_TEMP']


In [27]:
#drop temps

ENEM.drop(['EDU_PAI_TEMP', 'EDU_MAE_TEMP', 'QT_PESSOAS_CASA_TEMP','RENDA_MENSAL_TEMP'], axis=1, inplace=True)

In [28]:
print('total students considered:',ENEM.shape[0])

total students considered: 1110151


CENSUS - SCHOOL

In [29]:
#Taking only schools present at ALL DATABASES
C_SCHOOL = C_SCHOOL.loc[(C_SCHOOL.CO_ESCOLA.isin(ENEM.CO_ESCOLA)) & (C_SCHOOL.CO_ESCOLA.isin(C_CLASS.CO_ESCOLA)) &
                                                        (C_SCHOOL.CO_ESCOLA.isin(C_TEACHER.CO_ESCOLA))]
C_SCHOOL.isnull().sum().to_frame().T

,CO_ESCOLA,CO_MUNICIPIO,CO_UF,CO_DEPENDENCIA_ADM,IN_AGUA_INEXISTENTE,IN_ESGOTO_INEXISTENTE,IN_ENERGIA_INEXISTENTE,IN_LABORATORIO_INFORMATICA,IN_LABORATORIO_CIENCIAS,IN_SALA_ATENDIMENTO_ESPECIAL,IN_BIBLIOTECA,IN_SALA_LEITURA,IN_BANHEIRO,IN_BANHEIRO_PNE,QT_SALAS_UTILIZADAS,QT_EQUIP_TV,QT_EQUIP_DVD,QT_EQUIP_COPIADORA,QT_EQUIP_IMPRESSORA,QT_COMP_ALUNO,IN_INTERNET,IN_BANDA_LARGA,QT_FUNCIONARIOS,IN_ALIMENTACAO,IN_COMUM_MEDIO_MEDIO,IN_COMUM_MEDIO_INTEGRADO,IN_COMUM_MEDIO_NORMAL,IN_SALA_PROFESSOR,IN_COZINHA,IN_EQUIP_PARABOLICA,IN_ATIV_COMPLEMENTAR,IN_QUADRA_ESPORTES
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1025,48,572,0,0,0,0,0,0,0,0,0,3752


In [30]:
# IN = Mode
# QT - Mean
QT = C_SCHOOL.filter(regex=("^QT_*")).columns
IN = C_SCHOOL.filter(regex=("^IN_*")).columns

for i in IN:
    C_SCHOOL[i].fillna(C_SCHOOL[i].mean(), inplace = True)
for i in QT:
    C_SCHOOL[i].fillna(Counter(C_SCHOOL[i]).most_common()[0][0], inplace = True)


In [31]:
print('total schools considered:',C_SCHOOL.shape[0])

total schools considered: 21391


CENSUS - CLASS

In [32]:
C_CLASS = C_CLASS.loc[(C_CLASS.CO_ESCOLA.isin(C_SCHOOL.CO_ESCOLA))]
C_CLASS.isnull().sum().to_frame().T

,CO_ESCOLA,CO_TURMA,CO_MOD_ENSINO,IN_ETAPA_ENSINO,QT_MATRICULAS
0,0,0,0,0,0


CENSUS - TEACHER

In [33]:
C_TEACHER = C_TEACHER.drop_duplicates(['CO_PROFESSOR', 'CO_ESCOLA']) # We have multiple teachers for school due classes. It is importante just one record
C_TEACHER = C_TEACHER.loc[(C_TEACHER.CO_ESCOLA.isin(C_SCHOOL.CO_ESCOLA))]
C_TEACHER.isnull().sum().to_frame().T

,CO_PROFESSOR,CO_ESCOLA,CO_TURMA,IN_ESPECIALIZACAO,IN_MESTRADO,IN_DOUTORADO,ID_QUIMICA,ID_FISICA,ID_MATEMATICA,ID_BIOLOGIA,ID_SOCIOLOGIA,ID_LINGUA_LITERAT_PORTUGUESA,ID_LINGUA_LITERAT_INGLES,ID_LINGUA_LITERAT_ESPANHOL,ID_ARTES,ID_EDUCACAO_FISICA,ID_HISTORIA,ID_GEOGRAFIA,ID_FILOSOFIA,CO_AREA_OCDE_1,ID_LICENCIATURA_1,CO_AREA_OCDE_2,ID_LICENCIATURA_2,CO_AREA_OCDE_3,ID_LICENCIATURA_3
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,49900,49900,820412,820412,872291,872291


In [34]:
C_TEACHER.fillna({'IN_ESPECIALIZACAO':0, 'IN_MESTRADO':0, 'IN_DOUTORADO':0}, inplace =True)
print('total teachers considered:',C_TEACHER.shape[0])

total teachers considered: 879146


### New Features

ENEM

In [35]:
#Final Media, Target
ENEM['NU_NOTA_GERAL'] = (ENEM.NU_NOTA_CN + ENEM.NU_NOTA_CH + ENEM.NU_NOTA_LC + ENEM.NU_NOTA_MT + ENEM.NU_NOTA_REDACAO)/5

TEACHER CENSUS

In [36]:
C_TEACHER['TITULACAO'] = 0

C_TEACHER.loc[(C_TEACHER.IN_ESPECIALIZACAO==0.0) & (C_TEACHER.IN_MESTRADO==0.0)& 
             (C_TEACHER.IN_DOUTORADO ==0.0), 'TITULACAO'] = 0

C_TEACHER.loc[(C_TEACHER.IN_ESPECIALIZACAO==1.0) & (C_TEACHER.IN_MESTRADO==0.0)& 
             (C_TEACHER.IN_DOUTORADO ==0.0), 'TITULACAO'] = 1

C_TEACHER.loc[(C_TEACHER.IN_ESPECIALIZACAO==1.0) & (C_TEACHER.IN_MESTRADO==1.0)& 
             (C_TEACHER.IN_DOUTORADO ==0.0), 'TITULACAO'] = 2

C_TEACHER.loc[(C_TEACHER.IN_ESPECIALIZACAO==0.0) & (C_TEACHER.IN_MESTRADO==1.0)& 
             (C_TEACHER.IN_DOUTORADO ==0.0), 'TITULACAO'] = 2

C_TEACHER.loc[(C_TEACHER.IN_DOUTORADO ==1.0), 'TITULACAO'] = 3

#Rebuild level education information as indicator of unique level.
C_TEACHER['NU_GRADUACAO'] = 0
C_TEACHER['NU_ESPECIALIZACAO'] = 0
C_TEACHER['NU_MESTRADO'] = 0
C_TEACHER['NU_DOUTORADO'] = 0

C_TEACHER.loc[C_TEACHER.TITULACAO == 0, 'NU_GRADUACAO']= 1
C_TEACHER.loc[C_TEACHER.TITULACAO == 1, 'NU_ESPECIALIZACAO']= 1
C_TEACHER.loc[C_TEACHER.TITULACAO == 2, 'NU_MESTRADO']= 1
C_TEACHER.loc[C_TEACHER.TITULACAO == 3, 'NU_DOUTORADO']= 1


C_TEACHER['NU_GRADUACAO'] = C_TEACHER.groupby('CO_ESCOLA')['NU_GRADUACAO'].transform(sum)
C_TEACHER['NU_ESPECIALIZACAO'] = C_TEACHER.groupby('CO_ESCOLA')['NU_ESPECIALIZACAO'].transform(sum)
C_TEACHER['NU_MESTRADO'] = C_TEACHER.groupby('CO_ESCOLA')['NU_MESTRADO'].transform(sum)
C_TEACHER['NU_DOUTORADO'] = C_TEACHER.groupby('CO_ESCOLA')['NU_DOUTORADO'].transform(sum)



#Calculating weighted indice of teacher education
SUM= C_TEACHER.groupby('CO_ESCOLA')['TITULACAO'].transform(sum)
COUNT =  C_TEACHER.groupby('CO_ESCOLA')['TITULACAO'].transform('count')
C_TEACHER['TITULACAO'] = (SUM)/(COUNT*3)  


In [37]:
#In order to avoid count duplicates teacher who teachs in both subjects. Since the required training is the same
C_TEACHER['ID_LINGUA_ESTRANGEIRA']=0
C_TEACHER.loc[((C_TEACHER.ID_LINGUA_LITERAT_ESPANHOL==1)|
               (C_TEACHER.ID_LINGUA_LITERAT_INGLES==1)),'ID_LINGUA_ESTRANGEIRA']=1  

#In order to avoid count duplicates teacher who teachs in both subjects. Since we consindering as the same subject
C_TEACHER['ID_FISIO_SOCIO']=0
C_TEACHER.loc[((C_TEACHER.ID_FILOSOFIA==1)|(C_TEACHER.ID_SOCIOLOGIA==1)),'ID_FISIO_SOCIO']=1

C_TEACHER['OUTROS'] = 0
curr_teaching = ['ID_MATEMATICA', 'ID_FISICA', 'ID_QUIMICA', 'ID_HISTORIA', 'ID_BIOLOGIA', 'ID_GEOGRAFIA', 
            'ID_LINGUA_LITERAT_PORTUGUESA', 'ID_LINGUA_ESTRANGEIRA','ID_FISIO_SOCIO', 'ID_ARTES', 
            'ID_EDUCACAO_FISICA', 'OUTROS']

#C_TEACHER[curr_teaching].fillna(0, inplace=True)
C_TEACHER.loc[(C_TEACHER[curr_teaching]==0).all(axis=1), 'OUTROS'] =1

#Take out teacher who do not taught in subjects related of ENEM and secondary education
C_TEACHER = C_TEACHER.loc[C_TEACHER.OUTROS==0]
##

In [38]:
# How many teacher for each knowlodege area?

ciencias_humanas = ['ID_HISTORIA', 'ID_GEOGRAFIA','ID_FILOSOFIA', 'ID_SOCIOLOGIA']
linguagens_codigos = ['ID_LINGUA_LITERAT_PORTUGUESA', 'ID_LINGUA_LITERAT_ESPANHOL',
                      'ID_LINGUA_LITERAT_INGLES', 'ID_ARTES', 'ID_EDUCACAO_FISICA']
matematica = ['ID_MATEMATICA']
ciencias_natureza = ['ID_FISICA', 'ID_QUIMICA', 'ID_BIOLOGIA']

###
values = np.sum(C_TEACHER[ciencias_natureza].values == 1,1)
C_TEACHER['NU_CIENCIA_NATUREZA'] = [1 if x >= 1 else 0 for x in values]
C_TEACHER['NU_CIENCIA_NATUREZA']= C_TEACHER.groupby('CO_ESCOLA')['NU_CIENCIA_NATUREZA'].transform(sum)

values = np.sum(C_TEACHER[ciencias_humanas].values == 1,1)
C_TEACHER['NU_CIENCIAS_HUMANAS'] = [1 if x >= 1 else 0 for x in values]
C_TEACHER['NU_CIENCIAS_HUMANAS']= C_TEACHER.groupby('CO_ESCOLA')['NU_CIENCIAS_HUMANAS'].transform(sum)

values = np.sum(C_TEACHER[linguagens_codigos].values == 1,1)
C_TEACHER['NU_LINGUAGENS_CODIGOS'] = [1 if x >= 1 else 0 for x in values]
C_TEACHER['NU_LINGUAGENS_CODIGOS']= C_TEACHER.groupby('CO_ESCOLA')['NU_LINGUAGENS_CODIGOS'].transform(sum)

C_TEACHER['NU_MATEMATICA']= C_TEACHER.groupby('CO_ESCOLA')['ID_MATEMATICA'].transform(sum)

#Sum all domain teacher's number
C_TEACHER['NU_TEACHERS_DOMAIN'] = C_TEACHER['NU_MATEMATICA']+C_TEACHER['NU_LINGUAGENS_CODIGOS']+C_TEACHER[
    'NU_CIENCIAS_HUMANAS']+C_TEACHER['NU_CIENCIA_NATUREZA']

#update domain teacher's number normalized
C_TEACHER['NU_MATEMATICA']=C_TEACHER['NU_MATEMATICA']/C_TEACHER['NU_TEACHERS_DOMAIN']
C_TEACHER['NU_LINGUAGENS_CODIGOS'] = C_TEACHER['NU_LINGUAGENS_CODIGOS']/C_TEACHER['NU_TEACHERS_DOMAIN']
C_TEACHER['NU_CIENCIAS_HUMANAS'] = C_TEACHER['NU_CIENCIAS_HUMANAS']/C_TEACHER['NU_TEACHERS_DOMAIN']
C_TEACHER['NU_CIENCIA_NATUREZA'] = C_TEACHER['NU_CIENCIA_NATUREZA']/C_TEACHER['NU_TEACHERS_DOMAIN']

In [39]:
#Number of different schools that teachers work 
C_TEACHER['NU_ESCOLAS'] = C_TEACHER.groupby('CO_PROFESSOR')['CO_ESCOLA'].transform('count')
C_TEACHER['NU_ESCOLAS'] = C_TEACHER.groupby('CO_ESCOLA')['NU_ESCOLAS'].transform('mean')
#Number of licensed teachers
C_TEACHER['NU_LICENCIADOS']=C_TEACHER.groupby('CO_ESCOLA')['ID_LICENCIATURA_1'].transform(sum)
C_TEACHER['NU_TEACHERS']=C_TEACHER.groupby('CO_ESCOLA')['ID_LICENCIATURA_1'].transform('count')
C_TEACHER['NU_LICENCIADOS']=C_TEACHER['NU_LICENCIADOS']/C_TEACHER['NU_TEACHERS']

In [40]:
# How about the adequate background rate of teachers
FOR_HISTORIA = ['145F11', '225H01']
FOR_GEOGRAFIA = ['145F10', '443G04']
FOR_FISIO_SOCIO = ['226999','145F08', '312999', '145F24']
FOR_ARTES = ['210A01', '146D01']
#FOR_CIENCIAS_HUMANAS = [FOR_HISTORIA, FOR_GEOGRAFIA, FOR_FILOS_SOCIO]
##
FOR_LING_PORTUGUES = ['145F15','145F16', '220P01', '220L03', '220L01']
FOR_LING_ESTRANGEIRA = ['145F16','220E01', '220L03', '145F13']
FOR_EDUCACAO_FISICA = ['720E01']
##
FOR_MATEMATICA = ['461M01','145F18']
##
FOR_BIOLOGIA = ['145F01', '421B02']
FOR_QUIMICA = ['442Q01','421B09', '524999', '145F21']
FOR_FISICA = ['145F09', '421B01','441F01']
#FOR_CIENCIAS_NATUREZA = [biologia, quimica, fisica]
###
C_TEACHER['FOR_MATEMATICA']=0
C_TEACHER['FOR_FISICA']=0
C_TEACHER['FOR_QUIMICA']=0
C_TEACHER['FOR_HISTORIA']=0
C_TEACHER['FOR_BIOLOGIA']=0
C_TEACHER['FOR_GEOGRAFIA']=0
C_TEACHER['FOR_LING_PORTUGUES']=0
C_TEACHER['FOR_LING_ESTRANGEIRA']=0
C_TEACHER['FOR_FISIO_SOCIO']=0
C_TEACHER['FOR_ARTES']=0
C_TEACHER['FOR_EDUCACAO_FISICA']=0

### Set 1 for those who teaches and have adequate training for that
C_TEACHER.loc[(C_TEACHER.ID_MATEMATICA==1) & ((C_TEACHER.CO_AREA_OCDE_1.isin(FOR_MATEMATICA))|
              (C_TEACHER.CO_AREA_OCDE_2.isin(FOR_MATEMATICA))|
             (C_TEACHER.CO_AREA_OCDE_3.isin(FOR_MATEMATICA))), 'FOR_MATEMATICA'] = 1

C_TEACHER.loc[(C_TEACHER.ID_FISICA==1) & ((C_TEACHER.CO_AREA_OCDE_1.isin(FOR_FISICA))|
              (C_TEACHER.CO_AREA_OCDE_2.isin(FOR_FISICA))|
             (C_TEACHER.CO_AREA_OCDE_3.isin(FOR_FISICA))), 'FOR_FISICA'] = 1

C_TEACHER.loc[(C_TEACHER.ID_QUIMICA==1) & ((C_TEACHER.CO_AREA_OCDE_1.isin(FOR_QUIMICA))|
              (C_TEACHER.CO_AREA_OCDE_2.isin(FOR_QUIMICA))|
             (C_TEACHER.CO_AREA_OCDE_3.isin(FOR_HISTORIA))), 'FOR_QUIMICA'] = 1

C_TEACHER.loc[(C_TEACHER.ID_HISTORIA==1) & ((C_TEACHER.CO_AREA_OCDE_1.isin(FOR_HISTORIA))|
              (C_TEACHER.CO_AREA_OCDE_2.isin(FOR_HISTORIA))|
             (C_TEACHER.CO_AREA_OCDE_3.isin(FOR_HISTORIA))), 'FOR_HISTORIA'] = 1

C_TEACHER.loc[(C_TEACHER.ID_BIOLOGIA==1) & ((C_TEACHER.CO_AREA_OCDE_1.isin(FOR_BIOLOGIA))|
              (C_TEACHER.CO_AREA_OCDE_2.isin(FOR_BIOLOGIA))|
             (C_TEACHER.CO_AREA_OCDE_3.isin(FOR_BIOLOGIA))), 'FOR_BIOLOGIA'] = 1

C_TEACHER.loc[(C_TEACHER.ID_GEOGRAFIA==1) & ((C_TEACHER.CO_AREA_OCDE_1.isin(FOR_GEOGRAFIA))|
              (C_TEACHER.CO_AREA_OCDE_2.isin(FOR_GEOGRAFIA))|
             (C_TEACHER.CO_AREA_OCDE_3.isin(FOR_GEOGRAFIA))), 'FOR_GEOGRAFIA'] = 1

C_TEACHER.loc[(C_TEACHER.ID_LINGUA_LITERAT_PORTUGUESA==1) & ((C_TEACHER.CO_AREA_OCDE_1.isin(FOR_LING_PORTUGUES))|
              (C_TEACHER.CO_AREA_OCDE_2.isin(FOR_LING_PORTUGUES))|
             (C_TEACHER.CO_AREA_OCDE_3.isin(FOR_LING_PORTUGUES))), 'FOR_LING_PORTUGUES'] = 1

C_TEACHER.loc[(C_TEACHER.ID_LINGUA_ESTRANGEIRA==1) & ((C_TEACHER.CO_AREA_OCDE_1.isin(FOR_LING_ESTRANGEIRA))|
              (C_TEACHER.CO_AREA_OCDE_2.isin(FOR_LING_ESTRANGEIRA))|
             (C_TEACHER.CO_AREA_OCDE_3.isin(FOR_LING_ESTRANGEIRA))), 'FOR_LING_ESTRANGEIRA'] = 1   

               
C_TEACHER.loc[((C_TEACHER.ID_FILOSOFIA==1)|(C_TEACHER.ID_SOCIOLOGIA==1)) & ((C_TEACHER.CO_AREA_OCDE_1.isin(FOR_FISIO_SOCIO))|
              (C_TEACHER.CO_AREA_OCDE_2.isin(FOR_FISIO_SOCIO))|
             (C_TEACHER.CO_AREA_OCDE_3.isin(FOR_FISIO_SOCIO))), 'FOR_FISIO_SOCIO'] = 1   
                                                   
C_TEACHER.loc[(C_TEACHER.ID_ARTES==1) & ((C_TEACHER.CO_AREA_OCDE_1.isin(FOR_ARTES))|
              (C_TEACHER.CO_AREA_OCDE_2.isin(FOR_ARTES))|
             (C_TEACHER.CO_AREA_OCDE_3.isin(FOR_ARTES))), 'FOR_ARTES'] = 1  

C_TEACHER.loc[(C_TEACHER.ID_EDUCACAO_FISICA==1) & ((C_TEACHER.CO_AREA_OCDE_1.isin(FOR_EDUCACAO_FISICA))|
              (C_TEACHER.CO_AREA_OCDE_2.isin(FOR_EDUCACAO_FISICA))|
             (C_TEACHER.CO_AREA_OCDE_3.isin(FOR_EDUCACAO_FISICA))), 'FOR_EDUCACAO_FISICA'] = 1  

In [41]:
#Computing sum of teachers with adequate training for each school
adequate_training = ['FOR_MATEMATICA','FOR_FISICA','FOR_QUIMICA','FOR_HISTORIA','FOR_BIOLOGIA','FOR_GEOGRAFIA',
 'FOR_LING_PORTUGUES','FOR_LING_ESTRANGEIRA','FOR_FISIO_SOCIO','FOR_ARTES', 'FOR_EDUCACAO_FISICA']

for i in adequate_training:
    C_TEACHER[i] = C_TEACHER.groupby('CO_ESCOLA')[i].transform(sum)
    
#Computing sum of current classes taught, whatever if they have training
for i in curr_teaching: #defined earlier
    C_TEACHER[i] =  C_TEACHER.groupby('CO_ESCOLA')[i].transform(sum) 
    
#School Grain
C_TEACHER.drop_duplicates('CO_ESCOLA', inplace = True)

In [42]:
# Computing the rate of adequate training ((number of current with adequate training + Number those w/o traininig*0.5)/total) for each area)

C_TEACHER['IN_FOR_MAT'] = (C_TEACHER['FOR_MATEMATICA']+ (
    (C_TEACHER['ID_MATEMATICA'] - C_TEACHER['FOR_MATEMATICA'])*0.5))/C_TEACHER['ID_MATEMATICA']

C_TEACHER['IN_FOR_FIS'] = (C_TEACHER['FOR_FISICA']+ (
    (C_TEACHER['ID_FISICA'] - C_TEACHER['FOR_FISICA'])*0.5))/C_TEACHER['ID_FISICA']

C_TEACHER['IN_FOR_QUI'] = (C_TEACHER['FOR_QUIMICA']+ (
    (C_TEACHER['ID_QUIMICA'] - C_TEACHER['FOR_QUIMICA'])*0.5))/C_TEACHER['ID_QUIMICA']

C_TEACHER['IN_FOR_HIST'] = (C_TEACHER['FOR_HISTORIA']+ (
    (C_TEACHER['ID_HISTORIA'] - C_TEACHER['FOR_HISTORIA'])*0.5))/C_TEACHER['ID_HISTORIA']

C_TEACHER['IN_FOR_BIO'] = (C_TEACHER['FOR_BIOLOGIA']+ (
    (C_TEACHER['ID_BIOLOGIA'] - C_TEACHER['FOR_BIOLOGIA'])*0.5))/C_TEACHER['ID_BIOLOGIA']

C_TEACHER['IN_FOR_GEO'] = (C_TEACHER['FOR_GEOGRAFIA']+ (
    (C_TEACHER['ID_GEOGRAFIA'] - C_TEACHER['FOR_GEOGRAFIA'])*0.5))/C_TEACHER['ID_GEOGRAFIA']

C_TEACHER['IN_FOR_PORT'] = (C_TEACHER['FOR_LING_PORTUGUES']+ (
    (C_TEACHER['ID_LINGUA_LITERAT_PORTUGUESA'] - C_TEACHER['FOR_LING_PORTUGUES'])*0.5))/C_TEACHER['ID_LINGUA_LITERAT_PORTUGUESA']

C_TEACHER['IN_FOR_EST'] = (C_TEACHER['FOR_LING_ESTRANGEIRA']+ (
    (C_TEACHER['ID_LINGUA_ESTRANGEIRA'] - C_TEACHER['FOR_LING_ESTRANGEIRA'])*0.5))/C_TEACHER['ID_LINGUA_ESTRANGEIRA']

C_TEACHER['IN_FOR_FISIO_SOCIO'] = (C_TEACHER['FOR_FISIO_SOCIO']+ (
    (C_TEACHER['ID_FISIO_SOCIO'] - C_TEACHER['FOR_FISIO_SOCIO'])*0.5))/C_TEACHER['ID_FISIO_SOCIO']

C_TEACHER['IN_FOR_ARTES'] = (C_TEACHER['FOR_ARTES']+ (
    (C_TEACHER['ID_ARTES'] - C_TEACHER['FOR_ARTES'])*0.5))/C_TEACHER['ID_ARTES']

C_TEACHER['IN_FOR_ED_FISICA'] = (C_TEACHER['FOR_EDUCACAO_FISICA']+ (
    (C_TEACHER['ID_EDUCACAO_FISICA'] - C_TEACHER['FOR_EDUCACAO_FISICA'])*0.5))/C_TEACHER['ID_EDUCACAO_FISICA']

In [43]:
#Computing a new index for each school = Teacher Adequate Training
COLUMNS = C_TEACHER.filter(regex=("^IN_FOR_*")).columns
C_TEACHER.fillna({x:0 for x in COLUMNS}, inplace=True)
C_TEACHER['IN_FORM_DOCENTE']=C_TEACHER.filter(regex=("^IN_FOR_*")).sum(axis=1)/11

#Drop indices from each subject
#C_TEACHER.drop(COLUMNS, axis=1, inplace = True)

In [44]:
#drop all subjects and others that are no longer needed
C_TEACHER.drop(C_TEACHER.filter(regex=('^ID_*')).columns, axis=1, inplace = True)
others = ['CO_PROFESSOR','CO_TURMA', 'CO_AREA_OCDE_1','CO_AREA_OCDE_2', 'CO_AREA_OCDE_3','OUTROS',
          'IN_ESPECIALIZACAO', 'IN_MESTRADO', 'IN_DOUTORADO']
#C_TEACHER.drop(others, axis=1, inplace = True)
C_TEACHER.columns

Index(['CO_PROFESSOR', 'CO_ESCOLA', 'CO_TURMA', 'IN_ESPECIALIZACAO',
       'IN_MESTRADO', 'IN_DOUTORADO', 'CO_AREA_OCDE_1', 'CO_AREA_OCDE_2',
       'CO_AREA_OCDE_3', 'TITULACAO', 'NU_GRADUACAO', 'NU_ESPECIALIZACAO',
       'NU_MESTRADO', 'NU_DOUTORADO', 'OUTROS', 'NU_CIENCIA_NATUREZA',
       'NU_CIENCIAS_HUMANAS', 'NU_LINGUAGENS_CODIGOS', 'NU_MATEMATICA',
       'NU_TEACHERS_DOMAIN', 'NU_ESCOLAS', 'NU_LICENCIADOS', 'NU_TEACHERS',
       'FOR_MATEMATICA', 'FOR_FISICA', 'FOR_QUIMICA', 'FOR_HISTORIA',
       'FOR_BIOLOGIA', 'FOR_GEOGRAFIA', 'FOR_LING_PORTUGUES',
       'FOR_LING_ESTRANGEIRA', 'FOR_FISIO_SOCIO', 'FOR_ARTES',
       'FOR_EDUCACAO_FISICA', 'IN_FOR_MAT', 'IN_FOR_FIS', 'IN_FOR_QUI',
       'IN_FOR_HIST', 'IN_FOR_BIO', 'IN_FOR_GEO', 'IN_FOR_PORT', 'IN_FOR_EST',
       'IN_FOR_FISIO_SOCIO', 'IN_FOR_ARTES', 'IN_FOR_ED_FISICA',
       'IN_FORM_DOCENTE'],
      dtype='object')

In [45]:
C_TEACHER.shape

(21383, 46)

In [46]:
#JUSTS NUMBER OF ENROLLMENTS OF SCHOOLS
C_CLASS = C_CLASS[['CO_ESCOLA', 'QT_MATRICULAS']]
C_CLASS['QT_MATRICULAS'] = C_CLASS.groupby('CO_ESCOLA').transform('mean')
C_CLASS.drop_duplicates('CO_ESCOLA', inplace = True)

## ENEM STUDENT GRAIN 


In [47]:
# Merging Class and School
print('School',C_SCHOOL.shape)
CENSO = pd.merge(C_SCHOOL, C_CLASS, on = 'CO_ESCOLA', how = 'inner')
print('Class + School:', CENSO.shape[0])
# Next, merging with Teacher
CENSO = pd.merge(CENSO, C_TEACHER, on = 'CO_ESCOLA', how = 'inner')
print('School + Class + Teacher(taked off subject teachers out of context)',CENSO.shape)

School (21391, 32)
Class + School: 21391
School + Class + Teacher(taked off subject teachers out of context) (21383, 78)


In [48]:
#Now, all to Student Grain
print('ENEM',ENEM.shape)
ENEM_CENSO = pd.merge(ENEM, CENSO, on = 'CO_ESCOLA', how = 'inner')
print('FINAL (taked off schools with subject teachers out of context))',ENEM_CENSO.shape)

ENEM_CENSO.to_csv('STUDENT.csv')

ENEM (1110151, 16)
FINAL (taked off schools with subject teachers out of context)) (1070778, 93)
